In [1]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from scipy.spatial import distance_matrix

In [2]:
from sklearn.metrics import accuracy_score

In [3]:
data_train = pd.read_csv('data/spam/spam.train', delimiter=' ', header=None).values
data_train, target_train = data_train[:, 1:], data_train[:, 0]
data_test = pd.read_csv('data/spam/spam.test', delimiter=' ', header=None).values
data_test, target_test = data_test[:, 1:], data_test[:, 0]

In [4]:
clf = DecisionTreeClassifier(max_depth=3)

In [5]:
clf.fit(data_train, target_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [6]:
clf.tree_.max_depth

3

In [7]:
# accuracy_score(clf.predict(data_test), target_test)
accuracy_score(target_test, clf.predict(data_test))

0.75019888623707243

# CART with kNN on leaves

In [8]:
X = pd.read_csv('data/iris/iris.data', header=None).values

In [9]:
y = X[:, -1]
X = X[:, :-1]

In [10]:
class Tree(object):
    
    class Node(object):
        def __init__(self, index, splitter):
            self.index = index
            self.splitter = splitter
            self.left = None
            self.right = None

    def __init__(self):
        self.root = self.Node(None, None)
        self.depth = 0

    def __str__(self):
        if self.root is not None:
            self._print(self.root)

        return ''

    def _print(self, node):
        self.depth += 1
        if node is not None:
            print('\t' * (self.depth - 1), node.index.size, node.splitter)
            self._print(node.left)
            self._print(node.right)
        
        self.depth -= 1

    def insert(self, data):
        if self.root is None:
            self.root = self.Node(data)

        else:
            self._insert(self.root, data)

    def _insert(self, node, data):
        if node.data < data:
            if node.right is None:
                node.right = self.Node(data)
            else:
                self._insert(node.right, data)
                
        else:
            if node.left is None:
                node.left = self.Node(data)
            else:
                self._insert(node.left, data)

In [26]:
class DecisionTree(Tree):
    
    def __init__(self, criterion='gini', max_depth=100, min_samples_leaf=1, class_selector='kNN'):
        """
        criterion : string, optional (default="gini")
            The function to measure the quality of a split. Supported criteria are
            "gini" for the Gini impurity and "entropy" for the information gain.
        """
        super().__init__()
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.class_selector = class_selector
    
    def fit(self, X, y):
        self.X = X
        self.y_orig = y
        self.y_indxs = {n: i for i, n in enumerate(sorted(set(y)))}
        self.y_names = {i: n for n, i in self.y_indxs.items()}
        self.y = np.empty_like(y, dtype=np.int)
        for i in range(y.size):
            self.y[i] = self.y_indxs[y[i]]

        if self.criterion == 'gini':
            self._fit(self.root, np.arange(X.shape[0]))

#         elif self.criterion == 'entropy':
#             self._fit(self.root, np.arange(X.shape[0]), self.entropy)
            
        else:
            return "Error"
        
    def _fit(self, node, index, imp=1):
        self.depth += 1
        node.index = index
        if index.size > self.min_samples_leaf and \
            self.depth < self.max_depth and \
            imp > 1e-5:
            min_info = 10000 #2 * self.y.size
            for i, j in enumerate(self.X[index].T):
                argindx = np.argsort(j)
                ys = self.y[index][argindx]
                
                k = 0
                
                x_l = np.zeros(len(self.y_names))
                x_l[ys[k]] += 1
                s_l = np.sum(x_l)
                g_l = 0

                x_r = np.unique(ys[1:].tolist() + list(self.y_indxs.values()),
                                return_counts=True)[1] - 1
                s_r = np.sum(x_r)
                g_r = 1 - np.sum(x_r ** 2) / (s_r ** 2)
                
                n_l = 1
                n_r = ys.size - n_l
                
                crit = (n_l * g_l + n_r * g_r) / ys.size
                if crit < min_info:
                    min_info = crit
                    min_g_l = g_l
                    min_g_r = g_r
                    splitter = (i, j[argindx[k]])

                for y in ys[1:-1]:
                    k += 1
                    
                    n_l += 1
                    s_l += 1
                    x_l[y] += 1
                    g_l = 1 - (((s_l - 1) ** 2) * (1 - g_l) + 2 * x_l[y] - 1) / (s_l ** 2)
                    
                    n_r -= 1
                    s_r -= 1
                    x_r[y] -= 1
                    g_r = 1 - (((s_r + 1) ** 2) * (1 - g_r) - 2 * x_r[y] - 1) / (s_r ** 2)
                    
                    crit = (n_l * g_l + n_r * g_r) / ys.size
                    if crit < min_info:
                        min_info = crit
                        min_g_l = g_l
                        min_g_r = g_r
                        splitter = (i, j[argindx[k]])

            indL = index[self.X[index, splitter[0]] <= splitter[1]]
            indR = index[self.X[index, splitter[0]] > splitter[1]]
        
            node.splitter = splitter

            node.left = self.Node(None, None)
            self._fit(node.left, indL, min_g_l)

            node.right = self.Node(None, None)
            self._fit(node.right, indR, min_g_r)
        
        self.depth -= 1

    def predict(self, X):
        y = []
        for x in X:
            if self.class_selector == 'kNN':
                y += [self._predict(x, self.root, self.kNN)]
            
            elif self.class_selector == 'max':
                y += [self._predict(x, self.root, self.maxClass)]
            
        return np.array(y).ravel()
    
    def _predict(self, x, node, F):
        if node.splitter is None:
            return self.y_names[F(node.index, x)]
        
        if x[node.splitter[0]] <= node.splitter[1]:
            return self._predict(x, node.left, F)

        else:
            return self._predict(x, node.right, F)
    
    def maxClass(self, index, x):
        ys = np.unique(self.y[index], return_counts=True)
        return ys[0][ys[1].argmax()]
    
    def kNN(self, index, x, neighborsNum = 5):
        """
            index: numpy.ndarray - Nx1, leaf data index
            x: float64, predict data
            neighborsNum: number of neighbors
        """
        x = np.array([x])
        dist = distance_matrix(x, self.X[index])
            
        knn = self.y[index][dist.argsort(axis=1)[:, :neighborsNum]]
        ret = []
        for n in knn:
            tmp = np.unique(n, return_counts=True)
            ret += [tmp[0][tmp[1].argmax()]]
        
        return ret[0]

In [27]:
clf = DecisionTree(max_depth=3)

In [18]:
%time clf.fit(X, y)

CPU times: user 20.2 ms, sys: 1.32 ms, total: 21.5 ms
Wall time: 20.6 ms


In [19]:
print(clf)

 150 (2, 1.9)
	 50 None
	 100 (3, 1.7)
		 54 (2, 4.9)
			 48 (0, 4.9)
				 2 None
				 46 None
			 6 (2, 5.1)
				 4 (3, 1.5)
					 2 None
					 2 None
				 2 None
		 46 (2, 4.8)
			 3 None
			 43 None



In [20]:
pred = clf.predict(X)

In [21]:
accuracy_score(pred, y)

0.98666666666666669

In [28]:
%time clf.fit(data_train, target_train)

CPU times: user 14.3 s, sys: 166 ms, total: 14.4 s
Wall time: 14.6 s


In [29]:
pred = clf.predict(data_test)

In [30]:
accuracy_score(target_test, pred)

0.86863564041368335